**Validation Metrics**

This notebook accepts a trained model and outputs statistics of its performance.

* Performance on training, validation, and test set
* Plot our predictions versus assay output
* Overlap with conservation scores
* Overlap with CENTIPEDE motifs
* Scatter plot of our importance scores versus SHARPR scores
* ... Things I missed :)

**General Strategy for Importance Scores**

Let's try to get all types of importance scores into  nested dictionaries of the following spec:

(promoter, cell_type) -> chromosome -> position -> score


In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load in Experimental Data and SHARPR Predictions

In [6]:
# This loads in the experimental data
from mrpa_data import MrpaData
data = MrpaData()

In [37]:
# Loads in SHARPR activity scores
sharpr = {}
for promoter in data.promoters:
    for cell_type in data.cell_types:
        experiment_key = (promoter, cell_type)
        sharpr[experiment_key] = {}
        with open("../data/sharpr_scores/wig/{}_{}.wig".format(promoter, cell_type)) as wig:
            for line in wig:
                if line[0] == '#': continue
                chrom, start, end, score = line.strip().split()
                start, end = int(start), int(end)
                score = float(score)
                if end - start > 1: continue
                if chrom not in sharpr[experiment_key]: sharpr[experiment_key][chrom] = {}
                for pos in range(start, end):
                    assert pos not in sharpr[experiment_key][chrom]
                    sharpr[experiment_key][chrom][pos] = score

# Make Predictions from trained model

In [39]:
model = load_model() ####

predictions = {} # experiment -> chrom -> pos -> score
activity = {}    # experiment -> chrom -> pos -> deeplift / ism score
for position in positions:
    seq = get_seq(*position)  ###
    deeplift = get_deeplift(seq) ####

NameError: name 'model' is not defined

Our predictions versus assay output